# 주차요금계산
링크: https://school.programmers.co.kr/learn/courses/30/lessons/92341

## 내 풀이

In [1]:
def solution(fees, records):
    car_in = dict()
    car_out = dict()
    nums = []
    for r in records:
        time, num, status = r.split()
        if status == 'IN':
            car_in[num] = time # 입차 기록
            nums.append(num) # 차량 번호 리스트
        else:
            car_out[num] = time # 출차 기록
    
    def time_to_min(time): # 분 단위 변환 함수
        h, m = time.split(':')
        return int(h)*60 + int(m)
    
    def total_fee(minutes, fees): # 요금 계산 함수
        fee = fees[1]
        minutes -= fees[0]
        fee += minutes // fees[2] * fees[3]
        if minutes % fees[2] != 0:
            fee += fees[3]
        return fee
    
    fee_dict = dict(zip(sorted(nums), [0 for i in nums])) # 누적 요금
    for num in fee_dict.keys(): # 차량 번호 마다(정렬)
        if num in car_out.keys():
            minutes = time_to_min(car_out[num]) - time_to_min(car_in[num]) # 출차 - 입차
            fee_dict[num] = total_fee(minutes, fees)
        else:
            minutes = time_to_min('23:59') - time_to_min(car_in[num]) # 마감 - 입차
            fee_dict[num] = total_fee(minutes, fees)
    return list(fee_dict.values())

solution([180, 5000, 10, 600],\
    ["05:34 5961 IN", "06:00 0000 IN", "06:34 0000 OUT", "07:59 5961 OUT", "07:59 0148 IN", "18:59 0000 IN", "19:09 0148 OUT", "22:59 5961 IN", "23:00 5961 OUT"])

[-50200, 34400, -5200]

* 다시 입차한 경우가 계산 오류

## 내풀이 2

In [2]:
def solution(fees, records):
    def time_to_min(time): # 분 단위 변환 함수
        h, m = time.split(':')
        return int(h)*60 + int(m)
    
    def total_fee(minutes, fees): # 요금 계산 함수
        fee = fees[1]
        minutes -= fees[0]
        fee += minutes // fees[2] * fees[3]
        if minutes % fees[2] != 0:
            fee += fees[3]
        return fee
    
    time_dic = dict() # 입차 시간
    fee_dic = dict() # 누적 요금
    
    for r in records:
        time, num, status = r.split()
        time = time_to_min(time)
        if status == "IN": # 입차
            if num not in time_dic.keys() or time_dic[num] == 0: # 첫 입차거나 정산 끝났으면        
                time_dic[num] = time 
        else: # 출차
            minutes = time - time_dic[num]
            fee_dic[num] += total_fee(minutes, fees)
    for k, v in time_dic.items():
        if v != 0: # 미정산 된 차량이 있다면
            minutes = time_to_min('23:59') - v
            if k in fee_dic.keys():
                fee_dic[k] += total_fee(minutes, fees)
            else:
                fee_dic[k] = total_fee(minutes, fees)
    sorted(fee_dic.keys())
    
    return list(fee_dic.values())

solution([180, 5000, 10, 600],\
    ["05:34 5961 IN", "06:00 0000 IN", "06:34 0000 OUT", "07:59 5961 OUT", "07:59 0148 IN", "18:59 0000 IN", "19:09 0148 OUT", "22:59 5961 IN", "23:00 5961 OUT"])

KeyError: '0000'

* dict에 key가 있는 지 먼저 확인

In [3]:
def solution(fees, records):
    def time_to_min(time): # 분 단위 변환 함수
        h, m = time.split(':')
        return int(h)*60 + int(m)
    
    def total_fee(minutes, fees): # 요금 계산 함수
        fee = fees[1]
        minutes -= fees[0]
        fee += minutes // fees[2] * fees[3]
        if minutes % fees[2] != 0:
            fee += fees[3]
        return fee
    
    time_dic = dict() # 입차 시간
    fee_dic = dict() # 누적 요금
    
    for r in records:
        time, num, status = r.split()
        time = time_to_min(time)
        if status == "IN": # 입차
            if num not in time_dic.keys() or time_dic[num] == 0: # 첫 입차거나 정산 끝났으면        
                time_dic[num] = time 
        else: # 출차
            minutes = time - time_dic[num]
            if num in fee_dic.keys():
                fee_dic[num] += total_fee(minutes, fees)
            else:
                fee_dic[num] = total_fee(minutes, fees)
    for k, v in time_dic.items():
        if v != 0: # 미정산 된 차량이 있다면
            minutes = time_to_min('23:59') - v
            if k in fee_dic.keys():
                fee_dic[k] += total_fee(minutes, fees)
            else:
                fee_dic[k] = total_fee(minutes, fees)
    sorted(fee_dic.keys())
    
    return list(fee_dic.values())

solution([180, 5000, 10, 600],\
    ["05:34 5961 IN", "06:00 0000 IN", "06:34 0000 OUT", "07:59 5961 OUT", "07:59 0148 IN", "18:59 0000 IN", "19:09 0148 OUT", "22:59 5961 IN", "23:00 5961 OUT"])

[55600, 121200, 86200]

* 요금 정산 함수 수정

    재입차 했을 때, 이전 정산때 180분 미만으로 이용했으면 남은 시간만큼 무료

    => 누적 요금이 아닌, 누적 이용시간으로 계산. 요금 계산은 마지막에

In [4]:
def solution(fees, records):
    # 최종적으로 이용한 시간(분)으로 요금 계산
    
    limit = fees[0] # 기본 시간
    in_rec = dict() # 입차시간 str
    use_rec = dict() # 이용시간 int

    def t_to_m(time): # 분 단위 변환 함수
        h, m = time.split(':')
        return int(h)*60 + int(m)

    def fee_cal(minutes, fees): # 요금 계산 함수
        fee = fees[1]
        minutes -= fees[0]
        if minutes > 0:
            if minutes % fees[2] != 0:
                fee += (minutes // fees[2]+1) * fees[3]
            else:
                fee += (minutes // fees[2]) * fees[3]
        return fee
            
    for r in records:
        time, num, status = r.split()
        if status == 'IN': # 입차기록
            in_rec[num] = time
        else:
            if num not in use_rec.keys():
                use_rec[num] = t_to_m(time) - t_to_m(in_rec[num]) # 이용시간 계산
            else:
                use_rec[num] += t_to_m(time) - t_to_m(in_rec[num]) # 이용시간 누적
            in_rec[num] = -1 # 정산 완료
        
    for num, time in in_rec.items(): # 미출차 차량 자동 정산
        if time != -1:
            if num not in use_rec.keys():
                use_rec[num] = t_to_m('23:59') - t_to_m(in_rec[num]) # 이용시간 계산
            else:
                use_rec[num] += t_to_m('23:59') - t_to_m(in_rec[num]) # 이용시간 누적
                
    # 누적 이용 시간으로 요금 계산            
    nums = sorted(use_rec.keys())
    answer = []
    for num in nums:
        answer.append(fee_cal(use_rec[num], fees))
    
    
    return answer

solution([180, 5000, 10, 600],\
    ["05:34 5961 IN", "06:00 0000 IN", "06:34 0000 OUT", "07:59 5961 OUT", "07:59 0148 IN", "18:59 0000 IN", "19:09 0148 OUT", "22:59 5961 IN", "23:00 5961 OUT"])

[14600, 34400, 5000]